# intent classification
https://velog.io/@yelim421/Intent-Classification-o9jdf690

In [1]:
# !pip install konlpy

In [2]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from tensorflow.keras.models import load_model

In [3]:
df= pd.read_csv('./qa_final.csv.csv')
import pandas as pd
# qa_all= pd.read_csv('../kospi_진짜_진짜_최종.csv')

In [4]:
def word_clean(df):
    from hanspell import spell_checker
    import re
    from tqdm import tqdm
    CHANGE_FILTER = re.compile("([~!?\"':;&)(])") #제외할 문자
    okt = Okt()
    X_train = []
    for sentence in tqdm(df):
        sentence = re.sub(CHANGE_FILTER,'','{}'.format(sentence)) #문자 제외
#         sentence = sentence.upper()
        ok = spell_checker.check(sentence) # 맞춤법 확인
        sentence = ok.checked #바꾼 문자로 변경
        clean_words = []
        for word in okt.pos(sentence):
            if word[1] in ['Noun', 'Modifier', 'Alpha']: #명사, 관형사(가끔 나와서), 영어
                 clean_words.append(word[0]) #찾아진 것 중 단어만 추가
        sentence = ' '.join(clean_words)
        X_train.append(sentence)
    return X_train

In [5]:
okt = Okt()
max_len = 13
tokenizer = Tokenizer()

def question_processing(sentences):
    sentences = [sentences]
    inputs = []
    for sentence in sentences : 
        sentence = okt.morphs(sentence)
        encoded = tokenizer.texts_to_sequences([sentence])
        inputs.append(encoded[0])
    padded_inputs = pad_sequences(inputs, maxlen=max_len, padding='post')
    return padded_inputs

In [6]:
model_loaded = tf.keras.models.load_model('./intent_epo100_bs64.h5')
import pickle
with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [7]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(df['A'])

# y_train = idx_encode.transform(df['A'])

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {}
for key, value in label_idx.items():
    idx_label[value] = key

In [8]:
input_sentence = question_processing(word_clean(['신한은행의 올해쯤 되려나 싶은 per은?'])[0])
prediction = np.argmax(model_loaded.predict(input_sentence), axis = 1)
for p in prediction:print(idx_label[p])

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.52s/it]


1/1 [==============================] - 1s 1s/step
신한지주의 2022년도 PER은 4.5(배)입니다.


In [9]:
def pred(input_text):
    pred = []
    input_sentence = question_processing(word_clean([input_text])[0])
    prediction = np.argmax(model_loaded.predict(input_sentence), axis = 1)
    for p in prediction: pred.append(idx_label[p])
    return pred[0]

# def pred(input_text):
#     input_sentence = question_processing(word_clean([input_text]))
#     prediction = np.argmax(model.predict(input_sentence), axis = 1)
#     for p in prediction:
#         print(idx_label[p])

In [10]:
pred('신한은행의 올해쯤 되려나 싶은 per은?')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.56it/s]


1/1 [==============================] - 0s 68ms/step


'신한지주의 2022년도 PER은 4.5(배)입니다.'

# flask

In [ ]:
#-*- coding: utf-8 -*-
from flask import Flask, request, render_template, jsonify

app = Flask(__name__)
   
@app.route("/predict", methods=["POST"])
def predict():
    value = dict(request.form)
    for i in value:
        value = i
    word_clean(value)
    print(value)
    predictions = pred(value)
    print(predictions)
    # jsonify({"state": "SUCCESS", "prediction": prediction})
    return jsonify({"state": "SUCCESS", "prediction": predictions})

@app.route("/chat")
def chat():
#     user_input = request.form["user_input"]
#     prediction = pred(user_input)
    
    return render_template("chat.html")

@app.route("/")
def index():
    # return render_template("index.html")
    return render_template('index.html', message='안녕하세요! 저를 클릭해주세요')


app.run(host='192.168.10.196', port=5000) #http://192.168.10.196:5000/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.10.196:5000
Press CTRL+C to quit
192.168.10.196 - - [09/Feb/2023 20:22:13] "GET / HTTP/1.1" 200 -
192.168.10.196 - - [09/Feb/2023 20:22:13] "GET /static/img/ant.png HTTP/1.1" 304 -
0it [00:00, ?it/s]


{}


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.68it/s]

1/1 [==============================] - ETA: 0s

...

{}


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 62.59it/s]

1/1 [==============================] - 0s 49ms/step



192.168.10.173 - - [09/Feb/2023 20:22:27] "POST /predict HTTP/1.1" 200 -


SKC의 2022년도 ROE는 8.56(%)입니다.
